# Import

In [1]:
from functools import reduce
from pathlib import Path
import time
import datetime

import pandas as pd
import requests
from wmfdata import hive, spark
from wmfdata.utils import print_err, pd_display_all


# Parameters

In [2]:
# TSV file where metrics are or will be saved
FILENAME = "metrics/diversity_metrics.tsv"

# Metric month. The mediawiki_history snapshot must be from the metrics month or later.
last_month = datetime.date.today().replace(day=1) - datetime.timedelta(days=1)

METRICS_MONTH_TEXT = last_month.strftime("%Y-%m")
MEDIAWIKI_HISTORY_SNAPSHOT = last_month.strftime("%Y-%m")



In [3]:
datetime.date.today().replace(day=1) - datetime.timedelta(days=1)

datetime.date(2021, 7, 31)

# Preparation

In [4]:
# Convert our metrics month to all the formats we need and provide them in a dict
# so we can easily use them to format strings
metrics_month = METRICS_MONTH_TEXT
date_params = {
    "mediawiki_history_snapshot": MEDIAWIKI_HISTORY_SNAPSHOT,
    "metrics_month": str(metrics_month),
    #"metrics_prev_month": str(metrics_month - 1),
    #"metrics_month_start": str(metrics_month.start_time), 
    "metrics_month_first_day": str((datetime.date.today()- datetime.timedelta(days=31)).replace(day=1)),
    #"metrics_month_end": str(last_month),
    "metrics_month_last_day": str(last_month),
    "metrics_year": last_month.year,
    "metrics_cur_month" : last_month.month
}

# Load any previous results
try:
    old_metrics = (
        pd.read_csv(FILENAME, sep="\t", parse_dates = ["month"])
        .set_index("month")
    )
except FileNotFoundError:
    old_metrics = None
    
def prepare_query(filename):
    return (
        Path(filename)
        .read_text()
        .format(**date_params)
    )

# MariaDB and Hive query metrics


In [5]:
queries = {
    
    "global_south_pageviews": {
        "file": "queries/global_south_pageviews.hql",
        "engine": "hive"
    },
    "global_south_previews": {
        "file": "queries/global_south_previews.hql",
        "engine": "hive"
    },
    "global_north_previews": {
        "file": "queries/global_north_previews.hql",
        "engine": "hive"
    },
     "global_north_pageviews": {
        "file": "queries/global_north_pageviews.hql",
        "engine": "hive"
    }
       
}


for key, val in queries.items():
    query = prepare_query(val["file"])
    engine = val["engine"]
    print_err("Running {} on {}...".format(key, engine))
    
    if engine == "mariadb":
        result = mariadb.run(query)
    elif engine == "hive":
        result = spark.run(query)
    else:
        raise ValueError("Unknown engine specified.") 
    
    result = result.assign(month=lambda df: pd.to_datetime(df["month"]))
    val["result"] = result

Running global_south_pageviews on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running global_south_previews on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running global_north_previews on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running global_north_pageviews on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


# Combining and saving metrics

In [6]:
# Assemble list of result dataframes
results = [val["result"] for _, val in queries.items()]

# Merge them all, assuming that the month is the only common column
new_metrics = reduce(lambda l, r: pd.merge(l, r, how="outer"), results)

# Set the month as an index so combine_first works properly
new_metrics = new_metrics.set_index("month").sort_index()

# Add Metrics for Interactions
new_metrics['gs_interactions'] = new_metrics.apply(lambda x: x['gs_previews'] + x['gs_pageviews'], axis=1)
new_metrics['gn_interactions'] = new_metrics.apply(lambda x: x['gn_previews'] + x['gn_pageviews'], axis=1)


In [7]:
if old_metrics is None:
    metrics = new_metrics
else:
    metrics = new_metrics.combine_first(old_metrics)
    
pd_display_all(metrics.tail(10))

,gn_interactions,gn_pageviews,gn_previews,gs_interactions,gs_pageviews,gs_previews
month,,,,,,
2020-10-01,1.467844e+10,1.301749e+10,1.660959e+09,4.825703e+09,4.382705e+09,442998424.0
2020-11-01,1.532113e+10,1.355392e+10,1.767209e+09,4.793234e+09,4.350722e+09,442512210.0
2020-12-01,1.476698e+10,1.312603e+10,1.640947e+09,4.387849e+09,3.989396e+09,398453334.0
2021-01-01,1.611202e+10,1.427241e+10,1.839604e+09,4.515382e+09,4.097901e+09,417481377.0
2021-02-01,1.455890e+10,1.289671e+10,1.662185e+09,4.460031e+09,4.066807e+09,393223979.0
2021-03-01,1.593795e+10,1.413920e+10,1.798748e+09,4.854990e+09,4.417236e+09,437754445.0
2021-04-01,1.432737e+10,1.351259e+10,8.147820e+08,4.258869e+09,4.054406e+09,204463207.0
2021-05-01,1.521259e+10,1.358943e+10,1.623162e+09,4.593551e+09,4.164103e+09,429448469.0
2021-06-01,1.355870e+10,1.218313e+10,1.375564e+09,4.260437e+09,3.863987e+09,396450484.0


In [8]:
metrics.to_csv(FILENAME, sep="\t")